In [7]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=AUZWyThfl5cObDMRt97J8zMk12LPMS3Fww6OCvOb"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)


{"generated_at":"2025-04-21T11:08:12+00:00","rankings":[{"type_id":2,"name":"ATP","year":2025,"week":17,"gender":"men","competitor_rankings":[{"rank":1,"movement":0,"points":9350,"competitions_played":22,"competitor":{"id":"sr:competitor:49363","name":"Pavic, Mate","country":"Croatia","country_code":"HRV","abbreviation":"PAV"}},{"rank":1,"movement":0,"points":9350,"competitions_played":22,"competitor":{"id":"sr:competitor:51836","name":"Arevalo-Gonzalez, Marcelo","country":"El Salvador","country_code":"SLV","abbreviation":"ARE"}},{"rank":3,"movement":0,"points":7590,"competitions_played":25,"competitor":{"id":"sr:competitor:14898","name":"Heliovaara, Harri","country":"Finland","country_code":"FIN","abbreviation":"HEL"}},{"rank":4,"movement":0,"points":7590,"competitions_played":26,"competitor":{"id":"sr:competitor:637970","name":"Patten, Henry","country":"Great Britain","country_code":"GBR","abbreviation":"PAT"}},{"rank":5,"movement":0,"points":6460,"competitions_played":20,"competitor

In [9]:
import json
data = json.loads(response.text)
data

{'generated_at': '2025-04-21T11:08:12+00:00',
 'rankings': [{'type_id': 2,
   'name': 'ATP',
   'year': 2025,
   'week': 17,
   'gender': 'men',
   'competitor_rankings': [{'rank': 1,
     'movement': 0,
     'points': 9350,
     'competitions_played': 22,
     'competitor': {'id': 'sr:competitor:49363',
      'name': 'Pavic, Mate',
      'country': 'Croatia',
      'country_code': 'HRV',
      'abbreviation': 'PAV'}},
    {'rank': 1,
     'movement': 0,
     'points': 9350,
     'competitions_played': 22,
     'competitor': {'id': 'sr:competitor:51836',
      'name': 'Arevalo-Gonzalez, Marcelo',
      'country': 'El Salvador',
      'country_code': 'SLV',
      'abbreviation': 'ARE'}},
    {'rank': 3,
     'movement': 0,
     'points': 7590,
     'competitions_played': 25,
     'competitor': {'id': 'sr:competitor:14898',
      'name': 'Heliovaara, Harri',
      'country': 'Finland',
      'country_code': 'FIN',
      'abbreviation': 'HEL'}},
    {'rank': 4,
     'movement': 0,
     'p

In [13]:
Competitors_list = []
for comp in data["rankings"]:
    for competitor_ranking in comp.get("competitor_rankings", []):  
        competitor = competitor_ranking["competitor"]
        Competitors_list.append({
            "competitor_id": competitor.get("id"),
            "name": competitor.get("name"),
            "country": competitor.get("country"),
            "country_name": competitor.get("country_code"), 
            "abbreviation": competitor.get("abbreviation")
        })

In [15]:
import pandas as pd
df5=pd.DataFrame(Competitors_list)
df5.head()

,competitor_id,name,country,country_name,abbreviation
0,sr:competitor:49363,"Pavic, Mate",Croatia,HRV,PAV
1,sr:competitor:51836,"Arevalo-Gonzalez, Marcelo",El Salvador,SLV,ARE
2,sr:competitor:14898,"Heliovaara, Harri",Finland,FIN,HEL
3,sr:competitor:637970,"Patten, Henry",Great Britain,GBR,PAT
4,sr:competitor:87690,"Thompson, Jordan",Australia,AUS,THO


In [17]:

competitor_rankings_list = []
for comp in data["rankings"]:
    for competitor_ranking in comp.get("competitor_rankings", []):  
        competitor_rankings_list.append({
            "rank": competitor_ranking["rank"],
            "movement": competitor_ranking["movement"],
            "points": competitor_ranking["points"],
            "competitions_played": competitor_ranking["competitions_played"], 
            "competitor_id": competitor_ranking["competitor"]["id"]  
        })

In [19]:
import pandas as pd
df6=pd.DataFrame(competitor_rankings_list)
df6.head()

,rank,movement,points,competitions_played,competitor_id
0,1,0,9350,22,sr:competitor:49363
1,1,0,9350,22,sr:competitor:51836
2,3,0,7590,25,sr:competitor:14898
3,4,0,7590,26,sr:competitor:637970
4,5,0,6460,20,sr:competitor:87690


In [23]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    # port=3306
)

mycursor = mydb.cursor(buffered=True)
mycursor.execute("use project")
mycursor.execute("create table Competitors(competitor_id varchar(50) Primary key,name varchar(100) Not null,country varchar(100) Not null,country_name CHAR(3) Not null,abbreviation varchar(10) Not null)")

ProgrammingError: 1050 (42S01): Table 'competitors' already exists

In [25]:
df5['country_name'] = df5['country_name'].fillna('Unknown')
insert_query = "INSERT IGNORE INTO Competitors (competitor_id, name, country, country_name, abbreviation) VALUES (%s, %s, %s, %s, %s)"
for index, row in df5.iterrows():
    mycursor.execute(insert_query, (
        row['competitor_id'],
        row['name'],
        row['country'],
        row['country_name'],
        row['abbreviation']
    ))
mydb.commit()

In [31]:
mycursor.execute("""
    CREATE TABLE competitor_rankings (
        rank_id INT AUTO_INCREMENT PRIMARY KEY,
        rank INT NOT NULL,
        movement INT NOT NULL,
        points INT NOT NULL,
        competitions_played INT NOT NULL,
        competitor_id VARCHAR(50) NOT NULL,
        FOREIGN KEY (competitor_id) REFERENCES Competitors(competitor_id)
    )
""")

In [27]:
mycursor.execute("SELECT competitor_id FROM Competitors")
existing_ids = {row[0] for row in mycursor.fetchall()}
df6_filtered = df6[df6['competitor_id'].isin(existing_ids)]
insert_query = """
    INSERT INTO competitor_rankings (rank, movement, points, competitions_played, competitor_id) 
    VALUES (%s, %s, %s, %s, %s)
"""

for index, row in df6_filtered.iterrows():
    mycursor.execute(insert_query, (
        row['rank'],
        row['movement'],
        row['points'],
        row['competitions_played'],
        row['competitor_id']
    ))

mydb.commit()

In [29]:
#1.Get all competitors with their rank and points.
from tabulate import tabulate
mycursor.execute("SELECT rank,points,competitor_id FROM competitor_rankings")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+--------+----------+-----------------------+
|   rank |   points | competitor_id         |
+========+==========+=======================+
|      1 |     9440 | sr:competitor:49363   |
+--------+----------+-----------------------+
|      1 |     9440 | sr:competitor:51836   |
+--------+----------+-----------------------+
|      3 |     7590 | sr:competitor:14898   |
+--------+----------+-----------------------+
|      4 |     7590 | sr:competitor:637970  |
+--------+----------+-----------------------+
|      5 |     6460 | sr:competitor:87690   |
+--------+----------+-----------------------+
|      6 |     5760 | sr:competitor:95801   |
+--------+----------+-----------------------+
|      7 |     5700 | sr:competitor:15310   |
+--------+----------+-----------------------+
|      8 |     5610 | sr:competitor:36593   |
+--------+----------+-----------------------+
|      9 |     5520 | sr:competitor:52293   |
+--------+----------+-----------------------+
|     10 |     5125 | sr:competito

In [62]:
#2.Find competitors ranked in the top 5
mycursor.execute("""
SELECT
  cr.rank,
  cr.competitor_id,
  cr.points
FROM
 competitor_rankings cr
JOIN 
    competitors c ON cr.competitor_id = c.competitor_id
WHERE
 cr.rank<=5
order by cr.rank
 """)

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+--------+----------------------+----------+
|   rank | competitor_id        |   points |
+========+======================+==========+
|      1 | sr:competitor:49363  |     9440 |
+--------+----------------------+----------+
|      1 | sr:competitor:72376  |    10665 |
+--------+----------------------+----------+
|      1 | sr:competitor:51836  |     9440 |
+--------+----------------------+----------+
|      2 | sr:competitor:51387  |     8825 |
+--------+----------------------+----------+
|      3 | sr:competitor:14898  |     7590 |
+--------+----------------------+----------+
|      3 | sr:competitor:74949  |     7840 |
+--------+----------------------+----------+
|      4 | sr:competitor:637970 |     7590 |
+--------+----------------------+----------+
|      4 | sr:competitor:64496  |     6775 |
+--------+----------------------+----------+
|      5 | sr:competitor:87690  |     6460 |
+--------+----------------------+----------+
|      5 | sr:competitor:43860  |     5983 |
+--------+

In [60]:
#3.List competitors with no rank movement (stable rank)
mycursor.execute("""
SELECT
  cr.rank,
  c.name,
  cr.competitor_id,
  cr.movement,
  cr.points
FROM
 competitor_rankings cr
JOIN 
    competitors c ON cr.competitor_id = c.competitor_id
WHERE
 cr.movement = 0
 """)

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))


+--------+---------------------------------+-----------------------+------------+----------+
|   rank | name                            | competitor_id         |   movement |   points |
+========+=================================+=======================+============+==========+
|      1 | Pavic, Mate                     | sr:competitor:49363   |          0 |     9440 |
+--------+---------------------------------+-----------------------+------------+----------+
|      1 | Arevalo-Gonzalez, Marcelo       | sr:competitor:51836   |          0 |     9440 |
+--------+---------------------------------+-----------------------+------------+----------+
|      3 | Heliovaara, Harri               | sr:competitor:14898   |          0 |     7590 |
+--------+---------------------------------+-----------------------+------------+----------+
|      4 | Patten, Henry                   | sr:competitor:637970  |          0 |     7590 |
+--------+---------------------------------+-----------------------+--

In [54]:
#4.Get the total points of competitors from a specific country (e.g., Croatia)
mycursor.execute("""
SELECT 
    c.country,
    SUM(r.points) AS total_points
FROM 
    competitors c
JOIN 
    competitor_rankings r ON c.competitor_id = r.competitor_id
WHERE 
    c.country = 'Croatia'
""")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+-----------+----------------+
| country   |   total_points |
+===========+================+
| Croatia   |          16957 |
+-----------+----------------+


In [56]:
#5.Count the number of competitors per country
mycursor.execute("""
SELECT 
    country,
    COUNT(*) AS number_of_competitors
FROM 
    competitors
GROUP BY 
    country
""")

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))


+--------------------------+-------------------------+
| country                  |   number_of_competitors |
+==========================+=========================+
| Algeria                  |                       1 |
+--------------------------+-------------------------+
| Antigua and Barbuda      |                       1 |
+--------------------------+-------------------------+
| Argentina                |                      31 |
+--------------------------+-------------------------+
| Armenia                  |                       1 |
+--------------------------+-------------------------+
| Australia                |                      48 |
+--------------------------+-------------------------+
| Austria                  |                      11 |
+--------------------------+-------------------------+
| Belarus                  |                       1 |
+--------------------------+-------------------------+
| Belgium                  |                      13 |
+---------

In [66]:
#6.Find competitors with the highest points in the current week
mycursor.execute("""
SELECT 
    cr.rank,
    cr.points,
    c.name,
    cr.competitor_id,
    c.country
FROM 
    competitor_rankings cr
JOIN 
    competitors c ON cr.competitor_id = c.competitor_id
WHERE 
    cr.points = (SELECT MAX(points) FROM competitor_rankings)
""")

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+--------+----------+---------------------+---------------------+-----------+
|   rank |   points | name                | competitor_id       | country   |
+========+==========+=====================+=====================+===========+
|      1 |    10665 | Siniakova, Katerina | sr:competitor:72376 | Czechia   |
+--------+----------+---------------------+---------------------+-----------+


In [35]:
import pandas as pd
df6=pd.DataFrame(competitor_rankings_list)
df6.head()

,rank,movement,points,competitions_played,competitor_id
0,1,0,9440,23,sr:competitor:49363
1,1,0,9440,23,sr:competitor:51836
2,3,0,7590,26,sr:competitor:14898
3,4,0,7590,27,sr:competitor:637970
4,5,0,6460,20,sr:competitor:87690


In [19]:
import pandas as pd
df5=pd.DataFrame(Competitors_list)
df5.head()

,competitor_id,name,country,country_name,abbreviation
0,sr:competitor:49363,"Pavic, Mate",Croatia,HRV,PAV
1,sr:competitor:51836,"Arevalo-Gonzalez, Marcelo",El Salvador,SLV,ARE
2,sr:competitor:14898,"Heliovaara, Harri",Finland,FIN,HEL
3,sr:competitor:637970,"Patten, Henry",Great Britain,GBR,PAT
4,sr:competitor:87690,"Thompson, Jordan",Australia,AUS,THO


In [5]:
import os 
os.getcwd()

'D:\\'